In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import networkit as nk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from IPython.display import display, Math, Latex, Markdown
from tqdm.notebook import tqdm

from cluster_filter import cfilter, cassign

from External.ICT.calculate_ICT import calculate_ICT, calculate_sub_ICTs
from External.clustering import centers, k_means_pp
from External.generation import create_graph, load_image, sample_points_from_image
from External.plotting import plot_points, plot_graph, no_intersections
from patching import patch_together

# from External.create_k_nearest import patch_together

plt.style.use('animations.mplstyle')

In [5]:
# Hyperparameters
mode = "K_Nearest" # Graph construction mode
image_name = "image"
ICT_algorithm = "cluster_all"
metric = "euclidean" # metric for clustering
patching = "free" #free or fixed

name_of_image_folder = patching + "_patching/" + "run_1"

# Cluster rassignment
min_cluster_size = 12
small_behavior = "reassign" #reassign or remove

# image loading
n = number_of_nodes = 1000
Random = False


In [6]:
# Compute the position array
img = load_image(image_name)
position = np.array(sample_points_from_image(n,img,Random)).T
position = StandardScaler().fit_transform(position)

In [7]:
for k in range(2, 41):
    
    cluster_centers, cluster_labels = k_means_pp(k, position, metric=metric, return_labels=True)

    if small_behavior == "remove":
        cluster_centers, cluster_labels, (position, ) = cfilter(cluster_centers, cluster_labels, t=min_cluster_size, position_likes=[position])
        number_of_nodes = len(position)
    if small_behavior == "reassign":
        cluster_centers, cluster_labels = cassign(cluster_centers, cluster_labels, position, t=min_cluster_size)

    sub_ICTs, components = calculate_sub_ICTs(position, cluster_centers, cluster_labels, t=min_cluster_size, mode=mode)


    # plot the ICT forest
    ICT_forest = nk.graph.Graph(n=len(position), weighted=True)
    distances = pairwise_distances(position, position)
    for component, sub_ICT in zip(components, sub_ICTs):
        for u, v, w in sub_ICT.iterEdgesWeights():
            nodeA = component[u]
            nodeB = component[v]
            ICT_forest.addEdge(nodeA, nodeB, distances[nodeA, nodeB])

    ICT_forest.indexEdges()
    
#     G = patch_together(ICT_forest, position, bridges=4)

    G, f1, f2, good_edges = patch_together(ICT_forest, position, centrality_threshold = 5.5, distance_threshold=4.5, k=6)
    G.indexEdges()
    
    if patching == "fixed":

        ICT = calculate_ICT(G, algorithm_type=ICT_algorithm, cluster_centers=cluster_centers, zeros_stay_zeros=True, update_G=1.1, good_edges=good_edges)
        
    if patching == "free":

        ICT = calculate_ICT(G, algorithm_type=ICT_algorithm, cluster_centers=cluster_centers, zeros_stay_zeros=True, update_G=1.1)
        
    ICT.indexEdges()
    
    # Plot the ICT
    fig, axs = plt.subplots(1, 3, figsize=(18,6))
    plot_points(position, f"overwritten", axs[0], labels=np.array(cluster_labels))
    axs[0].get_legend().remove()
    plot_graph(ICT_forest, position, f"sub ICT forest with nodes ({k} clusters)", axs[0], node_size=0, edge_scale=0.5)
    
    plot_points(position[f2], f"overwritter", axs[1], labels=np.array(cluster_labels)[f2])
    axs[1].get_legend().remove()
    plot_graph(G, position, f"Patched graph with 'glue points'", axs[1], node_size=0, edge_scale=0.5)
    
    plot_graph(ICT, position, f"ICT without nodes ({k} clusters)", axs[2], node_size=0, edge_scale=0.5)
    name = str(k)
    plt.tight_layout()
    plt.savefig(f"Output/" + name_of_image_folder + "/"+ name.zfill(5) + ".png")

    plt.close()

sklearn is done: 0.14058279991149902
My own part is done: 0.0011856555938720703


  0%|          | 0/6 [00:00<?, ?it/s]

Final k: 11


create edgeId array:   0%|          | 0/1121 [00:00<?, ?it/s]

Calculate the lower bound for the weights:   0%|          | 0/1121 [00:00<?, ?it/s]

...

update Arr 689 -> 620
update Arr 620 -> 549
update Arr 549 -> 499
update Arr 499 -> 450
update Arr 450 -> 409
update Arr 409 -> 367
update Arr 367 -> 332
update Arr 332 -> 298
update Arr 298 -> 270
update Arr 270 -> 241
update Arr 241 -> 219
update Arr 219 -> 189
update Arr 189 -> 161
update Arr 161 -> 137
update Arr 137 -> 124
update Arr 124 -> 113
update Arr 113 -> 109
update Arr 109 -> 104
update Arr 104 -> 104
update Arr 104 -> 103


create edgeId array:   0%|          | 0/1888 [00:00<?, ?it/s]

Calculate the lower bound for the weights:   0%|          | 0/1888 [00:00<?, ?it/s]

Iteration over all nodes:   0%|          | 0/999 [00:00<?, ?it/s]

update Arr 1888 -> 1319
added 219 edges early
update Arr 1319 -> 1193
added 56 edges early
update Arr 1193 -> 1084
added 61 edges early
update Arr 1084 -> 1008
added 53 edges early
update Arr 1008 -> 999
added 11 edges early
added 521 edges early
